In [3]:
# CÉLULA 1: IMPORTAÇÕES
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import warnings

warnings.filterwarnings('ignore') # Apenas para limpar a saída de avisos
print("Bibliotecas importadas com sucesso!")

Bibliotecas importadas com sucesso!


In [4]:
# CÉLULA 2: CARREGAR, CRUZAR (MERGE) E DEFINIR O ALVO

# 1. Carregar AMBOS os arquivos originais
try:
    df_math = pd.read_csv('student_math_clean.csv')
    df_por = pd.read_csv('student_portuguese_clean.csv')
    print(f"Dados carregados: {len(df_math)} alunos de Mat. e {len(df_por)} alunos de Por.")
except FileNotFoundError:
    print("ERRO: Verifique se 'student_math_clean.csv' e 'student_portuguese_clean.csv' estão na pasta.")
    # Pare aqui se os arquivos não forem encontrados

# 2. Definir as colunas-chave para o cruzamento
# São as colunas demográficas/comportamentais idênticas em ambos os arquivos
chaves_aluno = [
    'school', 'sex', 'age', 'address_type', 'family_size', 'parent_status',
    'mother_education', 'father_education', 'mother_job', 'father_job',
    'school_choice_reason', 'guardian', 'travel_time', 'study_time',
    'class_failures', 'school_support', 'family_support', 'extra_paid_classes',
    'activities', 'nursery_school', 'higher_ed', 'internet_access',
    'romantic_relationship', 'family_relationship', 'free_time', 'social',
    'weekday_alcohol', 'weekend_alcohol', 'health'
]

# 3. Cruzar (Merge) os dataframes
# O 'suffixes' adiciona _mat e _por nas colunas que NÃO estão na chave
# (ou seja: student_id, absences, grade_1, grade_2, final_grade)
df_combinado = pd.merge(
    df_math, 
    df_por, 
    on=chaves_aluno, 
    suffixes=('_mat', '_por') # Ex: final_grade_mat, final_grade_por
)

print(f"\nCruzamento concluído. {len(df_combinado)} alunos encontrados em AMBAS as turmas.")

# 4. Criar a Nova Coluna-Alvo (y): 'em_risco_geral'
# Queremos saber se o aluno está em risco em MATEMÁTICA *OU* PORTUGUÊS
nota_de_corte = 10
em_risco_mat = (df_combinado['final_grade_mat'] < nota_de_corte)
em_risco_por = (df_combinado['final_grade_por'] < nota_de_corte)

# O operador | (OU) é a chave aqui
df_combinado['em_risco_geral'] = (em_risco_mat | em_risco_por).astype(int)

print("Coluna 'em_risco_geral' criada.")

# 5. Renomear o dataframe para o resto do script
df_processar = df_combinado.copy()

print("\nAmostra do novo alvo:")
print(df_processar[['final_grade_mat', 'final_grade_por', 'em_risco_geral']].head(10))
print("\nBalanceamento da classe-alvo:")
print(df_processar['em_risco_geral'].value_counts())

Dados carregados: 395 alunos de Mat. e 649 alunos de Por.

Cruzamento concluído. 162 alunos encontrados em AMBAS as turmas.
Coluna 'em_risco_geral' criada.

Amostra do novo alvo:
   final_grade_mat  final_grade_por  em_risco_geral
0                6               11               1
1                6               11               1
2               11               13               0
3                6               13               1
4               12               13               0
5               16               15               0
6               14               17               0
7               10               14               0
8               15               14               0
9               15               12               0

Balanceamento da classe-alvo:
em_risco_geral
0    110
1     52
Name: count, dtype: int64


In [5]:
# CÉLULA 3: PRÉ-PROCESSAMENTO (LIMPEZA DOS DADOS)

print("Iniciando pré-processamento...")

# 1. Mapeamento de colunas binárias (Sim/Não)
mapa_sim_nao = {'yes': 1, 'no': 0}
colunas_sim_nao = [
    'school_support', 'family_support', 'extra_paid_classes', 'activities', 
    'nursery_school', 'higher_ed', 'internet_access', 'romantic_relationship'
]
for col in colunas_sim_nao:
    df_processar[col] = df_processar[col].map(mapa_sim_nao)

# 2. Mapeamento de outras colunas binárias (manualmente)
df_processar['sex'] = df_processar['sex'].map({'F': 1, 'M': 0})
df_processar['address_type'] = df_processar['address_type'].map({'Urban': 1, 'Rural': 0})
df_processar['family_size'] = df_processar['family_size'].map({'Greater than 3': 1, 'Less than or equal to 3': 0})
df_processar['parent_status'] = df_processar['parent_status'].map({'Living together': 1, 'Apart': 0})

print("Colunas binárias ('sim/não', 'sexo', etc.) convertidas para 1 e 0.")

# 3. Mapeamento Ordinal (colunas que têm uma "ordem")
mapa_educacao = {'none': 0, 'primary education (4th grade)': 1, '5th to 9th grade': 2, 'secondary education': 3, 'higher education': 4}
df_processar['mother_education'] = df_processar['mother_education'].map(mapa_educacao)
df_processar['father_education'] = df_processar['father_education'].map(mapa_educacao)

mapa_travel = {'<15 min.': 1, '15 to 30 min.': 2, '30 min. to 1 hour': 3, '>1 hour': 4}
df_processar['travel_time'] = df_processar['travel_time'].map(mapa_travel)

mapa_study = {'<2 hours': 1, '2 to 5 hours': 2, '5 to 10 hours': 3, '>10 hours': 4}
df_processar['study_time'] = df_processar['study_time'].map(mapa_study)

print("Colunas ordinais (educação, tempo) convertidas para números.")

# 4. One-Hot Encoding (para colunas de categoria sem ordem)
colunas_one_hot = ['mother_job', 'father_job', 'school_choice_reason', 'guardian', 'school']
df_processar = pd.get_dummies(df_processar, columns=colunas_one_hot, drop_first=True)

print("Colunas categóricas (trabalho, escola) convertidas com One-Hot Encoding.")
print("\nPré-processamento concluído! Todas as colunas agora são numéricas.")

Iniciando pré-processamento...
Colunas binárias ('sim/não', 'sexo', etc.) convertidas para 1 e 0.
Colunas ordinais (educação, tempo) convertidas para números.
Colunas categóricas (trabalho, escola) convertidas com One-Hot Encoding.

Pré-processamento concluído! Todas as colunas agora são numéricas.


In [6]:
# CÉLULA 4: DEFINIR FEATURES (X) E TARGET (y)

# 'y' é a nossa nova coluna alvo combinada
y = df_processar['em_risco_geral']

# 'X' são todas as colunas, exceto as que dão a resposta
colunas_para_excluir = [
    'student_id_mat',    # ID não é preditivo
    'student_id_por',    # ID não é preditivo
    'final_grade_mat',   # Esta é a RESPOSTA (usar seria trapaça)
    'final_grade_por',   # Esta é a RESPOSTA (usar seria trapaça)
    'em_risco_geral'     # Esta é a nossa coluna-alvo (y)
]

X = df_processar.drop(columns=colunas_para_excluir)

# --- IMPORTANTE: Salvar a lista de features ---
# Seu 'servicos_ia.py' precisará desta nova lista!
lista_features = X.columns.tolist()
print(f"\nModelo será treinado com {len(lista_features)} colunas (features).")
print(lista_features)

# Salva a lista em um arquivo de texto
try:
    with open('features_modelo.txt', 'w') as f:
        for feature in lista_features:
            f.write(f"{feature}\n")
    print("\nLista de features salva em 'features_modelo.txt'")
except Exception as e:
    print(f"Erro ao salvar lista de features: {e}")


Modelo será treinado com 44 colunas (features).
['sex', 'age', 'address_type', 'family_size', 'parent_status', 'mother_education', 'father_education', 'travel_time', 'study_time', 'class_failures', 'school_support', 'family_support', 'extra_paid_classes', 'activities', 'nursery_school', 'higher_ed', 'internet_access', 'romantic_relationship', 'family_relationship', 'free_time', 'social', 'weekday_alcohol', 'weekend_alcohol', 'health', 'absences_mat', 'grade_1_mat', 'grade_2_mat', 'absences_por', 'grade_1_por', 'grade_2_por', 'mother_job_health', 'mother_job_other', 'mother_job_services', 'mother_job_teacher', 'father_job_health', 'father_job_other', 'father_job_services', 'father_job_teacher', 'school_choice_reason_home', 'school_choice_reason_other', 'school_choice_reason_reputation', 'guardian_mother', 'guardian_other', 'school_MS']

Lista de features salva em 'features_modelo.txt'


In [7]:
# CÉLULA 5: DIVIDIR DADOS (TREINO E TESTE)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.25,      # 25% dos dados para teste
    random_state=42,     # Garante que a divisão seja sempre a mesma
    stratify=y           # Garante proporção igual de 'em_risco_geral'
)

print(f"Total de dados: {len(X)}")
print(f"Dados de Treino (X_train): {len(X_train)}")
print(f"Dados de Teste (X_test): {len(X_test)}")

Total de dados: 162
Dados de Treino (X_train): 121
Dados de Teste (X_test): 41


In [8]:
# CÉLULA 6: TREINAR O MODELO

print("Iniciando treinamento do modelo Random Forest...")

rf_model = RandomForestClassifier(
    n_estimators=150,        
    max_depth=10,            
    class_weight='balanced', # Importante para balancear o novo 'em_risco_geral'
    random_state=42
)

# O "fit" é o comando de "aprender"
rf_model.fit(X_train, y_train)

print("Treinamento concluído.")

Iniciando treinamento do modelo Random Forest...
Treinamento concluído.


In [ ]:
# CÉLULA 7: VALIDAR O MODELO

print("\nAvaliando o modelo com os dados de Teste...")
y_pred = rf_model.predict(X_test)

precisao_geral = accuracy_score(y_test, y_pred)
print(f"\nAcurácia Geral: {(precisao_geral * 10):.2f}%")

print("\nRelatório de Classificação:")
print("---------------------------------")
print(classification_report(y_test, y_pred, target_names=['Sucesso (0)', 'Em Risco Geral (1)']))


Avaliando o modelo com os dados de Teste...

Acurácia Geral: 90.24%

Relatório de Classificação:
---------------------------------
                    precision    recall  f1-score   support

       Sucesso (0)       0.90      0.96      0.93        28
Em Risco Geral (1)       0.91      0.77      0.83        13

          accuracy                           0.90        41
         macro avg       0.90      0.87      0.88        41
      weighted avg       0.90      0.90      0.90        41



In [10]:
# CÉLULA 8: EXPORTAR O MODELO TREINADO (PARA PRODUÇÃO)

# O caminho aponta para a pasta principal do seu projeto
caminho_exportacao = "../modelo_risco_aluno.pkl" 

try:
    joblib.dump(rf_model, caminho_exportacao)
    print(f"\nModelo salvo com sucesso em: {caminho_exportacao}")
    print("Este arquivo agora pode ser usado pelo seu 'servicos_ia.py'.")
except Exception as e:
    print(f"Erro ao salvar o modelo: {e}")


Modelo salvo com sucesso em: ../modelo_risco_aluno.pkl
Este arquivo agora pode ser usado pelo seu 'servicos_ia.py'.
